# Homeworks

Base code for basic data we used during the whole week

In [53]:
#Import statements - To keed this in order
from sklearn.metrics import mean_absolute_error
from nltk.corpus import opinion_lexicon
import urllib.request, os, gzip
import  json
import random
import numpy # a powerfull module
from nltk.corpus import stopwords # We use it to remove the stopwords of the comments since they dont provide relevant info
from nltk.tokenize import sent_tokenize, word_tokenize # Divide text in sentences and then in words
from sklearn.linear_model import LinearRegression # sklearn is a machine learning toolkit (needs numpy, scipy and matplotlib)
from sklearn.linear_model import LogisticRegression # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# If I am not mistaken, logistic regresion is for booleans clasiffiers
from  sklearn.metrics import precision_score, recall_score # I think these are metrics to evaluate models form the slklearn library
from sklearn.naive_bayes import GaussianNB, BernoulliNB # To apply statistical models, not regressions(?)

In [2]:
%matplotlib inline
datadir = './data/'

def download_data(dataset_name, datadir):
    filename = 'reviews_%s_5.json' % dataset_name
    filepath = os.path.join(datadir, filename)
    if os.path.exists(filepath):
        print("Dataset %s has already been downloaded to %s" % (dataset_name, datadir))
    else:
        url = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/%s.gz' % filename
        urllib.request.urlretrieve(url, filepath + ".gz")
        with gzip.open(filepath + ".gz", 'rb') as fin:
            with open(filepath, 'wb') as fout:
                fout.write(fin.read())
        print("Downloaded dataset %s and saved it to %s" % (dataset_name, datadir))

dataset = "Baby"
download_data(dataset, datadir)

Dataset Baby has already been downloaded to ./data/


In [3]:
def  load_data (dataset_name, datadir):
    filepath = os.path.join(datadir, 'reviews_%s_5.json' % dataset_name)
    if not os.path.exists(filepath):
        download_data(dataset_name, datadir)
    data = []
    with open(filepath, 'r') as f:
        for line in f:                            # read file line by line
            item_hash = hash(line)                # we will use this later for partitioning our data 
            item = json.loads(line)               # convert JSON string to Python dict
            item['hash'] = item_hash              # add hash for identification purposes
            data.append(item)
    print("Loaded %d data for dataset %s" % (len(data), dataset_name))
    return data

# load the data...
baby = load_data(dataset, datadir)

Loaded 160792 data for dataset Baby


In [30]:
def partition_train_validation_test(data):
    # 60% : modulus is 0, 1, 2, 3, 4, or 5
    data_train = [item for item in data if item['hash']%10<=5]  
    # 20% : modulus is 6 or 7
    data_valid = [item for item in data if item['hash']%10 in [6,7]] 
    # 20% : modulus is 8 or 9
    data_test  = [item for item in data if item['hash']%10 in [8,9]] 
    print("Now we have", len(data_train), "training examples,", len(data_valid),
      "validation examples, and", len(data_test), "test examples")
    return data_train, data_valid, data_test
    
baby_train, baby_valid, baby_test = partition_train_validation_test(baby)




Now we have 96291 training examples, 32244 validation examples, and 32257 test examples


In [9]:
eng_stopwords = set(stopwords.words('english'))
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())

def my_tokenize(text):
    # split text into lower-case tokens, removing all-punctuation tokens and stopwords
    tokens = []
    for sentence in sent_tokenize(text):
        #Adds to the array and array with the words in lowercase, we add them if they are not stopwords and there is at least one letter in it
        tokens.extend(x for x in word_tokenize(sentence.lower()) #continues down...
                      if x not in eng_stopwords and any(i.isalpha() for i in x))# This extends the list by adding elements, it is different from append... see https://stackoverflow.com/questions/252703/difference-between-append-vs-extend-list-methods-in-python 
    return tokens

def pos_neg_fraction(text): # We recieve the raw text
    tokens = my_tokenize(text) # We tokenize the text first
    count_pos, count_neg = 0, 0
    for t in tokens:
        if t in positive_words:
            count_pos += 1
        if t in negative_words:
            count_neg += 1
    count_all = len(tokens) # this is because we need to be sure there is no 0 len sentence
    if count_all != 0:
        return count_pos/count_all, count_neg/count_all
    else:
        return 0., 0.
    
pos_example = 'This is a good, great, fantastic, amazing, wonderful, super product!!!'
neg_example = 'This is a bad, atrocious, terrible, dreadful, awful, abysmal product!!!'
print(pos_neg_fraction(pos_example))
print(pos_neg_fraction(neg_example))

(0.8571428571428571, 0.0)
(0.0, 0.8571428571428571)


In [12]:
def dataset_to_matrix(data):
    # data is a lot of text in {} that has reviwer name..comment...date...etc.. but all identified with text labels
    #in that sence data is an unidiminsional array
    # the item "atribute", we added it to the data to identify the sections of it
    return numpy.array([list(pos_neg_fraction(item['reviewText'])) for item in data])
# X_train with two columns and as many rows as there are examples in the data set. 
#The first column contains the fraction of positive words, 
#while the second column contains the fraction of negative words for each example.
X_train = dataset_to_matrix(baby_train)
most_pos, most_neg = numpy.argmax(X_train, axis=0) # find maximum ROW (axis 0).. through aaaaallll the data (OMG!)
# print the example with the highest fraction of positive words:
print("We found a fraction of %f %% positive words for example %d" % 
      (100.*X_train[most_pos, 0], most_pos))
print(baby_train[most_pos])
print("We found a fraction of %f %% negative words for example %d" %
      (100.*X_train[most_neg, 1], most_neg))
print(baby_train[most_neg])

We found a fraction of 100.000000 % positive words for example 10561
{'reviewerID': 'A2FPJGVT01TVVQ', 'asin': 'B000A88JYQ', 'reviewerName': 'S. Broderick "Sondra"', 'helpful': [0, 0], 'reviewText': 'work perfect', 'overall': 5.0, 'summary': 'Five Stars', 'unixReviewTime': 1404691200, 'reviewTime': '07 7, 2014', 'hash': -8116255050029596758}
We found a fraction of 100.000000 % negative words for example 24850
{'reviewerID': 'A1SLEYD29KEUW1', 'asin': 'B000WUD83O', 'reviewerName': 'ABDULLAH AL-FALAH', 'helpful': [0, 0], 'reviewText': 'too noisy', 'overall': 2.0, 'summary': 'Two Stars', 'unixReviewTime': 1404432000, 'reviewTime': '07 4, 2014', 'hash': -8656440585229339815}


In [13]:
def  dataset_to_targets (data):
    return numpy.array([item['overall'] for item in data])

Y_train = dataset_to_targets(baby_train)
print("Our feature matrix is two-dimensional and has shape", X_train.shape) # contains pos,neg fraction
print("Our target vector is one-dimensional and has shape", Y_train.shape) # containd sscore

Our feature matrix is two-dimensional and has shape (96291, 2)
Our target vector is one-dimensional and has shape (96291,)


## Day 1

In [15]:
lreg = LinearRegression().fit(X_train,Y_train)
print("The coefficient for the fpos variable is", lreg.coef_[0])
print("The coefficient for the fneg variable is", lreg.coef_[1])
print("The intercept is", lreg.intercept_)

The coefficient for the fpos variable is 3.25703026268183
The coefficient for the fneg variable is -5.593432938808693
The intercept is 4.004491648095601


In [17]:
#If the review contains 20% positive words (fpos==0.2) 
#but still no negative words (fneg==0), we would expect the following rating:
features = [[0.2, 0]]
expected_rating_A = lreg.predict(features)[0]
print("The expected rating is %f stars" % expected_rating_A)
# we can also compute this explicitly:
expected_rating_B = lreg.intercept_ + 0.2*lreg.coef_[0] + 0*lreg.coef_[1]
print("This is the same as %f stars" % expected_rating_B)
#However, if the review contains no positive words (fpos==0) but 20% negative words (fneg==0.2),
#we expect the following rating:
features = [[0, 0.2]]
expected_rating_A = lreg.predict(features)[0]
print("The expected rating is %f stars" % expected_rating_A)
# we can also compute this explicitly:
expected_rating_B = lreg.intercept_ + 0 * lreg.coef_[0] + 0.2 * lreg.coef_[1]
print("This is the same as %f stars" % expected_rating_B)

The expected rating is 4.655898 stars
This is the same as 4.655898 stars
The expected rating is 2.885805 stars
This is the same as 2.885805 stars


In [37]:
def predict_lreg(features):
    expected_rating = lreg.predict(features)
    expected_rating[expected_rating > 5.0] = 5.0
    expected_rating[expected_rating < 1.0] = 1.0
    return expected_rating

pred_train = predict_lreg(X_train)

In [39]:
mae_train = mean_absolute_error(pred_train, Y_train)
print("The mean absolute error on the training data is %f stars" % mae_train)

The mean absolute error on the training data is 0.827639 stars


#### - calculate the prediction for 100% pos, and 100% neg review

In [25]:
features_100pos=[[1,0]]
features_100neg=[[0,1]]
expected_rating_pos = lreg.predict(features_100pos)[0] # I think 0 is for linear as in kernel = {‘linear’, ‘rbf’, ‘poly’, ‘sigmoid’, ‘precomputed’}
expected_rating_neg = lreg.predict(features_100neg)[0]
print("The expected rating for 100 pos review is %f stars" % expected_rating_pos)
print("The expected rating  for 100 neg review is %f stars" % expected_rating_neg)
# This model needs a threshold to limit the starts between 0 and 5

The expected rating for 100 pos review is 7.261522 stars
The expected rating  for 100 neg review is -1.588941 stars


#### - Repeat this same process for "Apps for Android" dataset

In [29]:
###### Getting data####
android_dataset = "Apps_for_Android"
download_data(andorid_dataset, datadir)

Dataset Apps_for_Android has already been downloaded to ./data/


In [32]:
################Processing data############
# Load data from directory
apps= load_data(android_dataset,datadir)
#Partition data for training validation and tests
apps_train,apps_valid,apps_test = partition_train_validation_test(apps)
#We get the portions of positive and negative terms, omiting stopwords and puntuation marks (tokenizing words and sentences as well))
X_apps_train = dataset_to_matrix(apps_train)
app_most_pos, app_most_neg = numpy.argmax(X_apps_train, axis=0)
print(apps_train[app_most_pos])
print()
print(apps_train[app_most_neg])
Y_apps_train = dataset_to_targets(apps_train) # extract the overall item from each review

Loaded 752937 data for dataset Apps_for_Android
Now we have 451470 training examples, 150182 validation examples, and 151285 test examples
{'reviewerID': 'AWUDE6LOH5Y2Q', 'asin': 'B004DLPXAO', 'reviewerName': 'Amazon Customer', 'helpful': [0, 0], 'reviewText': 'Love it!', 'overall': 5.0, 'summary': 'Take you Kindle Library along whenever you have your phone!', 'unixReviewTime': 1404691200, 'reviewTime': '07 7, 2014', 'hash': -5981978766457409355}

{'reviewerID': 'A1HZYCWWJ53TS5', 'asin': 'B004HE5TAG', 'helpful': [0, 0], 'reviewText': 'boring', 'overall': 2.0, 'summary': 'Two Stars', 'unixReviewTime': 1405382400, 'reviewTime': '07 15, 2014', 'hash': -7538929081684597848}


In [36]:
####Linear regression
lreg_app = LinearRegression().fit(X_apps_train, Y_apps_train)
app_pos_features = [[0.2, 0]]
app_neg_features = [[0,0.2]]
expected_rating_pos = lreg.predict(app_pos_features)[0]
expected_rating_neg= lreg.predict(app_neg_features)[0]
print("The expected rating for a review with 20 porcet positive comments is %f stars" % expected_rating_pos)
print("The expected rating for a review with 20 porcet negative comments is %f stars" % expected_rating_neg)
#With this data we get sort of a similiar approximation

The expected rating for a review with 20 porcet positive comments is 4.655898 stars
The expected rating for a review with 20 porcet negative comments is 2.885805 stars


In [42]:
pred_train_apps = predict_lreg(X_apps_train)
mae_train_apps = mean_absolute_error(pred_train_apps, Y_apps_train)
print("The mean absolute error on the training data is %f stars" % mae_train_apps)
#We are getting even a higher error with a much more bigger dataset, funny, because it is bigger

The mean absolute error on the training data is 0.941797 stars


# Day 2

In [43]:
def discretize_targets(Y):
    return Y<= 3.0 # So this is a boolean, we are returning if the conditions is true
D_train = discretize_targets(Y_train) # We must remember that Y_train is a 1 dimension array with the scores
print("The training data contains %f %% dissatisfied customers" % (100.*D_train.mean()))
print(D_train[:10])

The training data contains 21.256400 % dissatisfied customers
[False False False False False False False False  True  True]


In [48]:
logreg = LogisticRegression().fit(X_train, D_train) # We use just the training data
# The predict_proba() method produces a matrix with two columns
# the first column contains the probability for the label being "false" (satisfied customer)
# the second column contains the probability for the label being "true" (dissatisfied customer)
# the sum of both columns is 1
# we select the second column with [:,1]
# [:,0] would select the first column
# [1,:] would select the second row
prob2_train = logreg.predict_proba(X_train)[:,1]
pred2_train = prob2_train > 0.5
max_prob2 = numpy.argmax(prob2_train)
min_prob2 = numpy.argmin(prob2_train)
def analyze_training_example_2(i):
    print("Training example number", i)
    print("True rating = %f stars" % Y_train[i])
    print("Expected to be dissatisfied:", pred2_train[i])
    print("Expected probability of being dissatisfied : %f" % prob2_train[i])
    print("Features = %f / %f" % (X_train[i,0], X_train[i,1]))
    print("Review text = %s" % baby_train[i]['reviewText'])
    
analyze_training_example_2(max_prob2)
print()
analyze_training_example_2(min_prob2)

Training example number 24850
True rating = 2.000000 stars
Expected to be dissatisfied: True
Expected probability of being dissatisfied : 0.999846
Features = 0.000000 / 1.000000
Review text = too noisy

Training example number 10561
True rating = 5.000000 stars
Expected to be dissatisfied: False
Expected probability of being dissatisfied : 0.000028
Features = 1.000000 / 0.000000
Review text = work perfect


In [51]:
precision2 = precision_score(D_train, pred2_train)
recall2 = recall_score(D_train, pred2_train)
print("For the default threshold (0.5) we get precision = %f "
      "and recall = %f" % (precision2, recall2))

For the default threshold (0.5) we get precision = 0.323503 and recall = 0.735832


In [54]:
nb = GaussianNB().fit(X_train, D_train)
prob3_train = nb.predict_proba(X_train)[:,1]
pred3_train = prob3_train>0.5
precision3 = precision_score(D_train, pred3_train)
recall3 = recall_score(D_train, pred3_train)
print("Now the precision is %f and the recall is %f" % (precision3, recall3))

Now the precision is 0.476183 and the recall is 0.155316


#### -Change the treshold from 0.5 to 0.2, and rerun the code.

In [56]:
new_pred2_train = prob2_train > 0.2
new_precision = precision_score(D_train, new_pred2_train)
new_recall =  recall_score(D_train, new_pred2_train)
print("For the default threshold (0.2) we get precision = %f "
      "and recall = %f" % (new_precision, new_recall))

For the default threshold (0.2) we get precision = 0.323503 and recall = 0.735832


#### - Give a commentary in plain English about how that changed precision and recall. What does that mean? What is now included that wasn't before? What part of it is good? What is bad from our Task perspective. Remember: our task was to identify Dissatisfied reviews.

Add features
Explain which features you chose, implement them, and write a commentary on your results

Feel welcome to use NLTK's built-in sentiment analyzer or any other research that you can find and understand


# DAy4